In [ ]:
%reload_ext autoreload
%autoreload 2

from concurrent.futures import ProcessPoolExecutor
from functools import partial
import h5py
import numpy as np
import time
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import re

from extra_data import by_index, open_run

from calibration import (
    dark_offset, parse_ids, find_proposal, get_mean_image,
    AzimuthalIntegration, ModuleRoiIntensity,
    ScatterPlot, SimpleImageViewer)

In [ ]:
# Config parameters

proposal = 900121
dark_run_nr = 51
dark_data_file = "/gpfs/exfel/data/scratch/kamile/lpd_timing_scan/lpd_low.h5"

timing_run_nr = 58

modules_roi = [32, 64, 0, 128]

modules = "0:16" # [0, 1, 2, ..., 15] # "Can also gives as comma separated : '1, 2, 14'"

pulse_ids_with_xray = "1:20:2"

# Delay motor device id and property
delay_src = "FXE_RR_SYS/TSYS/UTC-1-S3"
delay_prop = "backTrg3.delay.value"

# List of modules
module_numbers = parse_ids(modules)
module_numbers

In [ ]:
find_proposal(proposal, dark_run_nr)

# Quick look at timing run

In [ ]:
run = open_run(proposal, timing_run_nr)
run.info()

In [ ]:
def roi_intensity_wrapper(
    modno, proposal, run, dettype,
    window=1,
    rois=None,
    pulse_ids=None, 
    dark_run=None,
    use_normalizer=None,
    scan_variable=None):
    
    """
    modno: str, int
        Channel number between 0, 15
    proposal: str, int
        A proposal number, such as 2012, '2012', 'p002012', or a path such as
        '/gpfs/exfel/exp/SPB/201701/p002012'.
    run: str, int
        A run number such as 243, '243' or 'r0243'.
    dettype: (str) AGIPD, LPD
    window: (int) Moving average window size 
    pulse_ids: str
        For eg. ":" to select all pulses in a train
                "start:stop:step" to select indices with certain step size
                "1,2,3" comma separated pulse index to select specific pulses
                "1,2,3, 5:10" mix of above two
        Default: all pulses ":"
    rois: list
        In case of one roi: [x0, x1, y0, y1]
        For multiple rois: [[x0, x1, y0, y1], [x0, x1, y0, y1], ...]
    dettype: str
        "AGIPD", "LPD"
    dark_run: (numpy.ndarray) or dict optional
        dark_data shape (n_pulses, slow_scan, fast_scan)
        dark_run[module_number] of shape (n_pulses, slow_scan, fast_scan)
        Default: None,
        If provided dark data will be subtracted from images
    use_normalizer: tuple
        (source_name, property)
    
    scan_variable: tuple
        (karaboDeviceId, property)
    """
    
    mod_roi = ModuleRoiIntensity(modno, proposal, run, dettype, window=window)
    roi_intensity, roi_intensity_ma = mod_roi.eval_module_roi_intensity(
        rois=rois,
        pulse_ids=pulse_ids,
        dark_run=dark_run,
        use_normalizer=use_normalizer)

    if scan_variable is not None:
        src, prop = scan_variable
        mean_align, std_align, _ = mod_roi.plot_scan(src, prop)
        return roi_intensity, roi_intensity_ma, mean_align, std_align
    
    return roi_intensity, roi_intensity_ma

# Evaluate dark  for all modules in parallel and store in file


In [ ]:
# Store dark data in dark_data dictionary
# dark_data = {modno:data}
# Store dark data in dark_data dictionary
# dark_data = {modno:data}

dark_data = {}

_dark_eval = partial(dark_offset, 
                     proposal,
                     dark_run_nr,
                     pulse_ids=":",
                     dettype='LPD')

with ProcessPoolExecutor(max_workers=len(module_numbers)) as executor:
    for modno, ret in zip(module_numbers, executor.map(_dark_eval, module_numbers)):
        dark_data[modno] = ret

In [ ]:
print("Dark data available for modules :", dark_data.keys())
print("Dark data shape for module 1 : ", dark_data[1].shape)

In [ ]:
%matplotlib notebook

mod_to_view = 16  # Module number to plot
mem_cell_to_view = 2 # memory cell

fig, ax = plt.subplots(1,1)

rectangles = []
for roi in modules_roi:
    x, y = roi[2], roi[0]
    dx, dy = roi[3] - roi[2], roi[1] - roi[0]
    
    rectangles.append(
        patches.Rectangle(
            (x, y), dx, dy, 
            linewidth=2, 
            edgecolor='r', 
            facecolor='gray', 
            alpha=0.5))

for patch in rectangles:
    ax.add_patch(patch)

ax.imshow(dark_data[mod_to_view][mem_cell_to_view])

# Write dark data to file

In [ ]:
with h5py.File(dark_data_file, "w") as f:
    g = f.create_group(f"entry_1/instrument")
    for modno, data in dark_data.items():
        if data is not None:
            h = g.create_group(f"module_{modno}")
            h.create_dataset('data', data=data)

### Read dark data from files and store in dictionary dark_run

In [ ]:
dark_run = {}

import re
def iterate(name, node):
    if isinstance(node, h5py.Dataset):
        m = re.search("(.+)module_(.+)/data", name)
        if m is not None:
            dark_run[int(m.group(2))] = node[:]

with h5py.File(dark_data_file, 'r') as f:
    f.visititems(iterate)

dark_run.keys()

# Evaluate dark subtracted ROI intensities

In [ ]:
# Store roi intensities in roi_intensities dictionary

roi_intensities = {}

_roi_intensity_eval = partial(roi_intensity_wrapper, 
    proposal=proposal, 
    run=timing_run_nr, 
    dettype="LPD", 
    rois=modules_roi,
    pulse_ids=pulse_ids_with_xray,
    dark_run=dark_run,
    scan_variable=(delay_src, delay_prop))


with ProcessPoolExecutor(max_workers=len(dark_run.keys())) as executor:
    for modno, ret in zip(
        dark_run.keys(), executor.map(
            _roi_intensity_eval, dark_run.keys())):
        roi_intensities[modno] = ret

# Plot ROI intensities as a function of scan parameter

In [ ]:
_, _, mean_align, std_align = roi_intensities[16]
mean_align['scan_data']

# Plot delay data and ROI intensities

In [ ]:
x_offset = 0 #Provide a number to offset the xaxis so that xaxis has better visualization.

for modno in sorted(dark_run.keys(), key=lambda x: int(x)):
    if roi_intensities[modno] is not None:
        fig = ScatterPlot(title=f'Module {modno}',
                          xlabel=f"Scan variable ({delay_src}/{delay_prop})",
                          ylabel="Mean ROI intensity",
                          legend='Pulse index',
                          drop_down_label="ROI")
        _, _, mean_align, std_align = roi_intensities[modno]
        fig.setData(
                mean_align['scan_data'] - x_offset,
                mean_align['roi_intensity'],
                yerror=std_align['roi_intensity'])
        fig.show()

# Browse thorugh dark subtracted images

In [ ]:
config = dict(
    run_folder=find_proposal(proposal, timing_run_nr),
    dark_data=dark_data_file
)

w = SimpleImageViewer("LPD", config)
w.control_panel()